# Modules

In [1]:
from pdathome.constants import global_constants as gc
from pdathome.preprocessing import preprocess_filtering_gait

# Process data

In [3]:
for subject_id in gc.participant_ids.PD_IDS + gc.participant_ids.HC_IDS:
    preprocess_filtering_gait(subject_id)